In [1]:
import pandas as pd
import numpy as np
import warnings
import zipfile
import regex as re
warnings.filterwarnings('ignore')

In [2]:
zf = zipfile.ZipFile('/Users/borja/IronHack/Proyecto-Sharks_Attacks/data/Archive.zip')


archivo = zf.open('attacks.csv')


data_original = pd.read_csv(archivo, encoding='latin1')
pd.set_option('display.max_columns', None)

In [3]:
data_original.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [4]:
data = data_original.copy()

In [5]:
data.shape

(25723, 24)

In [6]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [7]:
duplicados = data[data.duplicated()] #borramos las filas publicadas
len(duplicados)

19411

In [8]:
data = data.drop_duplicates() #tenemos un nuevo DF con 19.411 filas borradas
data.shape

(6312, 24)

In [9]:
data.isna().sum()

Case Number                  2
Date                        10
Year                        12
Type                        14
Country                     60
Area                       465
Location                   550
Activity                   554
Name                       220
Sex                        575
Age                       2841
Injury                      38
Fatal (Y/N)                549
Time                      3364
Species                   2848
Investigator or Source      27
pdf                         10
href formula                11
href                        10
Case Number.1               10
Case Number.2               10
original order               3
Unnamed: 22               6311
Unnamed: 23               6310
dtype: int64

In [10]:
data.reset_index(drop=True, inplace=True)
filas_de_nulos = data.isna().mean(axis=1) > 0.6

In [11]:
data[filas_de_nulos].shape #solo hay 10 filas con nulos. no borramos por no ser significativo

(10, 24)

In [12]:
data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [13]:
data.columns = [c.lower().strip().replace(' ', '_') for c in data.columns]

In [14]:
data.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_(y/n)', 'time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number.1', 'case_number.2', 'original_order', 'unnamed:_22',
       'unnamed:_23'],
      dtype='object')

# Case Number

In [15]:
data.case_number.value_counts()

0               8
1920.00.00.b    2
1966.12.26      2
2014.08.02      2
1990.05.10      2
               ..
1999.09.05      1
1999.09.10      1
1999.09.16      1
1999.09.18      1
xx              1
Name: case_number, Length: 6287, dtype: int64

In [16]:
data.case_number = data.case_number.str.strip()

In [17]:
def filtrar_fechas(texto):
    # Definir el patrón de fecha: año.mes.día
    texto = str(texto)
    patron_fecha = r'\b\d{4}\.\d{2}\.\d{2}\b'
    
    
    
    # Buscar todas las fechas que coincidan con el patrón en el texto
    fechas = re.findall(patron_fecha, texto)

    # Unir todas las fechas encontradas en una sola cadena de texto
    fechas_string = ' '.join(fechas) 

    
    return fechas_string

In [18]:
data.case_number = data.case_number.apply(filtrar_fechas) #aplicamos la función anterior y corregimos fechas

In [19]:
data.case_number.value_counts()

              142
1950.00.00     13
1942.00.00     11
1957.00.00     11
1960.00.00     10
             ... 
1995.06.23      1
1995.06.24      1
1995.06.30      1
1995.07.00      1
0005.00.00      1
Name: case_number, Length: 5200, dtype: int64

In [20]:
#todo lo que venga con fechas anteriores a 1970 lo eliminamos. 
#No vamos a trabajar con fecha anteriores a los último 50 años
def filtrar_fechas2(texto):
    pattern = r'(\d{4})\.(\d{2})\.(\d{2})'
    match = re.search(pattern, texto)
    if match:
        year, month, day = match.groups()
        if int(year) < 1970:
            return np.nan
        if month == '00':
            month = '01'
        if day == '00':
            day = '01'
        return f'{year}.{month}.{day}'
    else:
        return np.nan


In [21]:
data.case_number = data.case_number.apply(filtrar_fechas2)

In [22]:
data.case_number.value_counts()

1970.01.01    9
1972.01.01    5
2003.10.05    5
1995.07.28    5
2001.04.12    5
             ..
2007.12.09    1
2007.12.07    1
2007.11.18    1
2007.11.08    1
2003.10.14    1
Name: case_number, Length: 2821, dtype: int64

In [23]:
data.case_number = pd.to_datetime(data.case_number,format='%Y-%m-%d')

In [24]:
data.case_number.head()

0   2018-06-25
1   2018-06-18
2   2018-06-09
3   2018-06-08
4   2018-06-04
Name: case_number, dtype: datetime64[ns]

# Date

In [25]:
data.date.value_counts()

1957                          11
1942                           9
1956                           8
1958                           7
1950                           7
                              ..
01-Dec-1994                    1
Reported      10-Dec-1994      1
11-Dec-1994                    1
13-Dec-1994                    1
1845-1853                      1
Name: date, Length: 5433, dtype: int64

In [26]:
data.date = data.date.str.strip()
data.date = data.date.str.lstrip('Reported ')

In [27]:
def verificar_formato(fecha):
    fecha = str(fecha)
    pattern = re.compile(r'\b\d{2}-[a-zA-Z]{3}-\d{4}\b')
    if pattern.fullmatch(fecha):
        return fecha
    else:
        return np.nan

In [28]:
data.date = data.date.apply(verificar_formato)

In [29]:
data.date = pd.to_datetime(data.date,format='%Y-%m-%d')

ValueError: time data "25-Jun-2018" at position 0 doesn't match format specified

In [ ]:
data.date.value_counts()

In [ ]:
data[['case_number','date']].head(15)

In [ ]:
indice_nulos = data[data.date.isna()].index
data.loc[indice_nulos,'date'] = data.loc[indice_nulos,'case_number']

In [ ]:
data[['case_number','date']].head(15)

In [ ]:
data.date = data.case_number
data.date = data['case_number.1']
data.date = data['case_number.2']

In [ ]:
data.head()

# Type

In [ ]:
data.type.value_counts()

In [ ]:
data.type = data.type.str.replace('Boatomg', 'Boat')

In [ ]:
data.type = data.type.str.replace('Boating', 'Boat')

In [ ]:
data.type = data.type.str.replace('Questionable', 'Other')

In [ ]:
data.type = data.type.str.replace('Invalid', 'Other')

In [ ]:
data.type.value_counts()

# Country

In [ ]:
len(data.country.unique())

In [ ]:
data.country = data.country.str.upper()
data.country = data.country.str.replace(r'[(\)\-\?]+', ' ')
data.country = data.country.str.replace('CEYLON  ','')
data.country = data.country.replace(['NORTH SEA', 'RED SEA / INDIAN OCEAN', 'BRITISH WEST INDIES', 'ASIA', 'COAST OF AFRICA', 'WEST INDIES', 'CENTRAL PACIFIC', 'RED SEA', 'MID ATLANTIC OCEAN', 'SOUTH PACIFIC OCEAN', 'BETWEEN PORTUGAL & INDIA', 'MEDITERRANEAN SEA'], np.nan)
data.country = data.country.str.strip()
len(data.country.unique())

In [ ]:
data.country.value_counts()

# Area

In [ ]:
len(data.area.unique())

In [ ]:
data.area = data.area.str.upper()
data.area = data.area.str.strip()

In [ ]:
data.area.value_counts()

# Year

In [ ]:
len(data.year.unique())

In [ ]:
data.year.value_counts()

In [ ]:
data.year.notnull().head(20)

In [ ]:
data.info(memory_usage='deep')

In [ ]:
#eliminamos valores nulos de las dos columnas donde todos los valores son NaN
#def reemplazar_nulos_con_unknown(data, columna):
   # data[columna] = data[columna].fillna('unknown')
   # return data

In [ ]:
#data = reemplazar_nulos_con_unknown(data, 'Unnamed: 22')

In [ ]:
#data = reemplazar_nulos_con_unknown(data, 'Unnamed: 23')

In [ ]:
#data.head()

In [ ]:
#data.info(memory_usage='deep')

In [ ]:
#df[df-date.isin()]